In [1]:
import os
from google.colab import drive
#mounting it to google drive.
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys

#The files must be in a called folder GPT-anvit
sys.path.append('/content/drive/MyDrive/GPT-anvit')

In [3]:
import torch
from torch.utils.data import DataLoader
from torch import nn, optim
from tqdm.notebook import tqdm
from model import GPTModel
from gpt_config import GPTConfig
from data_utils import train_tokenizer, load_tokenizer, load_data
from utils import generate_text
import time


In [4]:
#data shoudl be in a folder called data inside GPT-anvit
data_files = ["/content/drive/MyDrive/GPT-anvit/data/finance_corpus.txt"]  # your data
tokenizer_path = "tokenizer_finance.json"
if not os.path.exists(tokenizer_path):
    tokenizer = train_tokenizer(data_files, vocab_size=10000, save_path=tokenizer_path)
else:
    tokenizer = load_tokenizer(tokenizer_path)

pad_id = tokenizer.token_to_id("[PAD]")


Tokenizer trained and saved to tokenizer_finance.json


In [5]:
block_size = 256
vocab_size = 10000
train_dataset, val_dataset = load_data(data_files, tokenizer, block_size=block_size)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

config = GPTConfig(vocab_size=vocab_size, max_len=1024, n_layer=6, n_head=8, n_embd=512)
model = GPTModel(config).to(device)

In [7]:
total_start_time = time.time()

epochs = 10
learning_rate = 3e-4
criterion = nn.CrossEntropyLoss(ignore_index=pad_id)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(1, epochs + 1):
    start_time = time.time()

    model.train()
    total_loss = 0.0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch}/{epochs}", leave=False)
    for inputs, targets in progress_bar:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.view(-1, outputs.size(-1)), targets.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())

    end_time = time.time()
    epoch_time = end_time - start_time

    print(f"\nEpoch {epoch} completed in {epoch_time:.2f} seconds.")
    print(f"Total training loss: {total_loss:.4f}")

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch} Training Loss: {avg_loss:.4f}")

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs.view(-1, outputs.size(-1)), targets.view(-1))
            val_loss += loss.item()
    avg_val_loss = val_loss / len(val_loader)
    perplexity = torch.exp(torch.tensor(avg_val_loss))
    print(f"Epoch {epoch} Validation Loss: {avg_val_loss:.4f}, Perplexity: {perplexity:.4f}")

total_end_time = time.time()
total_duration = total_end_time - total_start_time
print(f"Total training time: {total_duration:.2f} seconds")


Epoch 1/10:   0%|          | 0/2177 [00:00<?, ?it/s]


Epoch 1 completed in 117.72 seconds.
Total training loss: 9803.3064
Epoch 1 Training Loss: 4.5031
Epoch 1 Validation Loss: 3.6324, Perplexity: 37.8030


Epoch 2/10:   0%|          | 0/2177 [00:00<?, ?it/s]


Epoch 2 completed in 116.62 seconds.
Total training loss: 7371.1051
Epoch 2 Training Loss: 3.3859
Epoch 2 Validation Loss: 3.1496, Perplexity: 23.3261


Epoch 3/10:   0%|          | 0/2177 [00:00<?, ?it/s]

...


Epoch 8 completed in 116.50 seconds.
Total training loss: 4979.7775
Epoch 8 Training Loss: 2.2874
Epoch 8 Validation Loss: 2.5066, Perplexity: 12.2633


Epoch 9/10:   0%|          | 0/2177 [00:00<?, ?it/s]


Epoch 9 completed in 116.03 seconds.
Total training loss: 4820.2969
Epoch 9 Training Loss: 2.2142
Epoch 9 Validation Loss: 2.4654, Perplexity: 11.7685


Epoch 10/10:   0%|          | 0/2177 [00:00<?, ?it/s]


Epoch 10 completed in 115.83 seconds.
Total training loss: 4683.7879
Epoch 10 Training Loss: 2.1515
Epoch 10 Validation Loss: 2.4305, Perplexity: 11.3651
Total training time: 1213.34 seconds


In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/GPT-anvit/finance_gpt.pth")
print("Model saved to finance_gpt.pth")

#model.load_state_dict(torch.load("finance_gpt.pth", map_location=device))


Model saved to finance_gpt.pth


In [8]:
prompt = "Which stocks are promising?"

generated = generate_text(model, tokenizer, prompt, max_length=500, top_k=20, device=device)
print(generated)


Which stocks are promising ? An imal Health is a global medical device company with a clear , having a clear competitive edge in the medical space that operates . It carries a Zacks Rank # 3 ( Hold ) at present . The Zacks Consensus Estimate for its current year has increased 1 . 5 % over the past month to $ 1 . 95 per share . Currently , this Finance stock offers a 2 . 26 % dividend yield from the past 52 weeks . Want the latest recommendations from Zacks Investment Research ? Today , you can download 7 Best Stocks for the Next 30 Days . Click to get this free report Novo Nordisk A / S ( NVO ) : Free Stock Analysis Report Eli Lilly and Company ( LLY ) : Free Stock Analysis Report Article 38 23 : Trump tariff threat ens , drug makers are doing well in focus , with GLP - 1 s as FDA dec ep tive tariffs Trump ' s tariffs ( Reuters ) - Eli Lilly and Novo Nordisk ( NVO ) - Nasdaq said on Thursday that it expects its GLP - 1 sales to rise in 2025 . " The healthcare conglomerate is also incre